In [ ]:
#HELPERS

def freeze_base_model_weights(model):
    sd = model.state_dict()
    grad_layers = ["lm_head.weight"]
    c=0
    for k in sd.keys():
        p = sd[k]
        if k in grad_layers:
            p.requires_grad = True
        else: 
            c+=1
            
            p.requires_grad = False
def process_arxiv_src(path, out_dir, article_ids):
    with open(path, "wt") as f:
        pass
    
def save_article_list(sampled_articles,out_path):
    with open(out_path, "wt") as f:
        f.writelines(sampled_articles)

        
def req_article(article_id):
    link = 'http://zzz.cs.cornell.edu:8080/txt_tex?' + article_id
    with libreq.urlopen(link) as url:
      r = url.read()
    return r


def text_to_inputs(ds, tokenizer, text_column, label_column):
    def preprocess_text(batch):
        text_encoding=tokenizer(''.join(batch[text_column]), max_length=1024)
        batch_inputs = text_encoding["input_ids"]
        batch_attentions = text_encoding["attention_mask"]
        
        labels_encoding=tokenizer(''.join(batch[label_column]), max_length=512)
        batch_labels = labels_encoding["input_ids"]
        return {"input_ids": batch_inputs, "attention_mask": batch_attentions, "labels": batch_labels}
    
    return ds.map(preprocess_text, num_proc=4)



def get_text_and_id(arxiv_dir, sample_size=0):
    file_names = []
    file_paths = []
    for root, dirs, files in os.walk(arxiv_dir, topdown=False):
        print(root, dirs, files)
        if root == arxiv_dir:
            for name in files:
                file_names.append(name[:-4])
                file_paths.append(os.path.join(arxiv_dir,name))

    if sample_size:
        sample_indices = np.random.randint(low = 0, high = len(file_names), size = sample_size)   
    else:
        sample_indices = range(len(file_names))
    txt_dataset = {"id": [], "text": []}
    for i in sample_indices:
        name = file_names[i]
        path = file_paths[i]
        with open(path, "r") as f:
            article = f.read()
            txt_dataset["id"].append(name)
            txt_dataset["text"].append(article)
            
    return datasets.Dataset.from_dict(txt_dataset)


def save_list(strings, out_path):
    strings = [strings] if isinstance(strings, str) else strings
    with open(out_path, "wt") as f:
        for i in strings:
            f.write(i + "\n\n\n")
            

class ArxivWikiTrainer(Seq2SeqTrainer):

    def compute_metrics(p):
        m = dataset.load_metric("bertscore")
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        preds = np.argmax(preds, axis=1)
        print("here")
        return {"accuracy": metric_acc.compute(predictions=preds, references=p.label_ids)["accuracy"]}

    """
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0], device=model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        print(loss)
        return (loss, outputs) if return_outputs else loss
    """